In [1]:
import pandas as pd
from fuzzywuzzy import fuzz

### Importing Cleaned Data

In [2]:
urlTransfer = 'https://raw.githubusercontent.com/edb-313/Football_players_trasfer_value/main/Transfers/Transfers_cleaned.csv'
urlStats = 'https://raw.githubusercontent.com/edb-313/Football_players_trasfer_value/main/StatsData/StatsData_Cleaned.csv'
urlContractDate = 'https://raw.githubusercontent.com/edb-313/Football_players_trasfer_value/main/ContractDate/ContractDates_cleaned.csv'

transfers_df = pd.read_csv(urlTransfer)
stats_df = pd.read_csv(urlStats)
contractDate_df = pd.read_csv(urlContractDate)

In [3]:
transfers_df.head()

,player_name,age,position,club_involved_name,transfer_period,fee_cleaned,year
0,Ángel Di María,22.0,Right Winger,Real Madrid,Summer,33000000.0,2010
1,Ramires,23.0,Central Midfield,Chelsea,Summer,22000000.0,2010
2,Rafik Halliche,23.0,Centre-Back,Fulham,Summer,2500000.0,2010
3,Ariza Makukula,29.0,Centre-Forward,Manisaspor,Summer,1700000.0,2010
4,David Luiz,23.0,Centre-Back,Chelsea,Winter,25000000.0,2010


In [4]:
stats_df.head()

,Player,Nation,Pos,Age,cum_MP,cum_Min,cum_Gls,cum_Ast,cum_PK,cum_CrdY,cum_CrdR,cum_GlsxMin,cum_AstxMin,cum_G+AxMin,Season
0,Quim,pt POR,GK,31,30.0,2639.0,0.0,0.0,0.0,1.0,1.0,0.00,0.00,0.00,2007
1,Rui Costa,pt POR,MF,35,29.0,2382.0,5.0,5.0,0.0,6.0,0.0,0.19,0.19,0.38,2007
2,Kostas Katsouranis,gr GRE,DFMF,28,27.0,2320.0,2.0,1.0,0.0,9.0,0.0,0.08,0.04,0.12,2007
3,Léo,br BRA,DF,32,27.0,2323.0,0.0,3.0,0.0,2.0,0.0,0.00,0.12,0.12,2007
4,Óscar Cardozo,py PAR,FW,24,29.0,2283.0,13.0,1.0,3.0,2.0,0.0,0.51,0.04,0.55,2007


### Merging transfers_df and stats_df

In [5]:
#If transfer was done in summer, we have to use stats from previous year
transfers_df['stats_year_summer'] = transfers_df['year']-1

In [6]:
# create a mask to filter summer transfers
summer_mask = transfers_df['transfer_period'] == 'Summer'

# create a mask to filter winter transfers
winter_mask = transfers_df['transfer_period'] == 'Winter'

# perform join for summer transfers
summer_join = pd.merge(transfers_df[summer_mask], stats_df, left_on=['player_name', 'stats_year_summer'], right_on=['Player', 'Season'], how='inner')

# perform join for winter transfers
winter_join = pd.merge(transfers_df[winter_mask], stats_df, left_on=['player_name', 'year'], right_on=['Player', 'Season'], how='inner')

# concatenate the results
merged_df = pd.concat([summer_join, winter_join], ignore_index=True)

In [7]:
merged_df

,player_name,age,position,club_involved_name,transfer_period,fee_cleaned,year,stats_year_summer,Player,Nation,...,cum_Min,cum_Gls,cum_Ast,cum_PK,cum_CrdY,cum_CrdR,cum_GlsxMin,cum_AstxMin,cum_G+AxMin,Season
0,Ángel Di María,22.0,Right Winger,Real Madrid,Summer,33000000.0,2010,2009,Ángel Di María,ar ARG,...,1841.142857,3.428571,7.142857,0.000000,6.428571,0.571429,0.157143,0.308571,0.465714,2009
1,Ramires,23.0,Central Midfield,Chelsea,Summer,22000000.0,2010,2009,Ramires,br BRA,...,1945.000000,4.000000,3.000000,0.000000,2.000000,0.000000,0.190000,0.140000,0.320000,2009
2,Fábio Coentrão,23.0,Left-Back,Real Madrid,Summer,30000000.0,2011,2010,Fábio Coentrão,pt POR,...,1594.400000,1.066667,3.200000,0.000000,6.400000,1.066667,0.048000,0.149333,0.197333,2010
3,Roberto,25.0,Goalkeeper,Real Zaragoza,Summer,8600000.0,2011,2010,Roberto,es ESP,...,2227.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,2010
4,Axel Witsel,23.0,Defensive Midfield,Zenit S-Pb,Summer,40000000.0,2012,2011,Axel Witsel,be BEL,...,2281.000000,1.000000,3.000000,1.000000,5.000000,0.000000,0.040000,0.120000,0.160000,2011
5,Javi García,25.0,Defensive Midfield,Man City,Summer,20200000.0,2012,2011,Javi García,es ESP,...,1883.142857,1.571429,0.571429,0.000000,8.714286,0.285714,0.071429,0.022857,0.094286,2011
6,Lorenzo Melgarejo,23.0,Left Winger,Kuban Krasnodar,Summer,5000000.0,2013,2012,Lorenzo Melgarejo,py PAR,...,1870.000000,2.000000,3.000000,0.000000,4.000000,1.000000,0.100000,0.140000,0.240000,2012
7,Nolito,26.0,Left Winger,Celta de Vigo,Summer,4000000.0,2013,2012,Nolito,es ESP,...,666.666667,4.333333,2.333333,0.000000,1.666667,0.000000,0.523333,0.130000,0.653333,2012
8,Jan Oblak,21.0,Goalkeeper,Atlético Madrid,Summer,16000000.0,2014,2013,Jan Oblak,si SVN,...,1351.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,2013
9,Ezequiel Garay,27.0,Centre-Back,Zenit S-Pb,Summer,6000000.0,2014,2013,Ezequiel Garay,ar ARG,...,2384.571429,4.000000,0.000000,0.000000,2.714286,0.000000,0.148571,0.000000,0.148571,2013


For some players, the name isn't exactly equal in transfers_df and stats_df. We will now use fuzzywuzzy to find the best match for each transfer_df['player_name'], and then join stats_df and transfer_df based on the best match

In [8]:
# create a list of player names in 'stats_df'
stats_player_names = stats_df['Player'].unique()

# create a function to find the best match for a given player name
def find_best_match(player_name):
    best_ratio = -1
    best_match = None
    for stats_player_name in stats_player_names:
        ratio = fuzz.token_sort_ratio(player_name, stats_player_name)
        if ratio > best_ratio:
            best_ratio = ratio
            best_match = stats_player_name
    return best_match

# create a column in 'transfers_df' with the best matching player name from 'stats_df'
transfers_df['best_match'] = transfers_df['player_name'].apply(find_best_match)

# perform join for summer transfers
summer_join = pd.merge(transfers_df[summer_mask], stats_df, left_on=['best_match', 'stats_year_summer'], right_on=['Player', 'Season'], how='inner')

# perform join for winter transfers
winter_join = pd.merge(transfers_df[winter_mask], stats_df, left_on=['best_match', 'year'], right_on=['Player', 'Season'], how='inner')

# concatenate the results
merged_fuzzy_df= pd.concat([summer_join, winter_join], ignore_index=True)

Now that we have merged_fuzzy_df, let's check what additional joins it was able to do in respect to the initial merge

In [9]:
#remove best_match column as it is present only on merged_fuzzy_df and not in merged_df
merged_fuzzy_df.drop(['best_match'],axis=1,inplace=True)

In [10]:
#Concatenate the two merged dfs
double_merged_df = pd.concat([merged_df, merged_fuzzy_df], ignore_index=True)

#Drop the duplicates to remain only with the rows present only in merged_fuzzy_df
double_merged_df.drop_duplicates(keep=False, inplace=True)

In [11]:
#Display double_merged_df
double_merged_df

,player_name,age,position,club_involved_name,transfer_period,fee_cleaned,year,stats_year_summer,Player,Nation,...,cum_Min,cum_Gls,cum_Ast,cum_PK,cum_CrdY,cum_CrdR,cum_GlsxMin,cum_AstxMin,cum_G+AxMin,Season
47,Daniel Wass,23.0,Central Midfield,Évian,Summer,1200000.0,2012,2011,Axel Witsel,be BEL,...,2281.000000,1.000000,3.000000,1.000000,5.000000,0.000000,0.040000,0.120000,0.160000,2011
49,Émerson Conceição,26.0,Left-Back,Trabzonspor,Summer,1600000.0,2012,2011,Emerson,br BRA,...,2093.000000,0.000000,1.000000,0.000000,6.000000,1.000000,0.000000,0.040000,0.040000,2011
52,Lazar Markovic,20.0,Right Winger,Liverpool,Summer,25000000.0,2014,2013,Lazar Marković,rs SRB,...,1659.000000,5.000000,5.000000,0.000000,2.000000,0.000000,0.270000,0.270000,0.540000,2013
56,Stefan Mitrović,24.0,Centre-Back,SC Freiburg,Summer,1500000.0,2014,2013,Nemanja Matić,rs SRB,...,1467.714286,2.142857,0.000000,0.000000,6.571429,0.285714,0.134286,0.000000,0.134286,2013
62,Mehdi Carcela-González,27.0,Right Winger,Granada CF,Summer,4000000.0,2016,2015,Mehdi Carcela,ma MAR,...,822.000000,2.000000,3.000000,0.000000,1.000000,0.000000,0.220000,0.330000,0.550000,2015
66,Konstantinos Mitroglou,29.0,Centre-Forward,Marseille,Summer,15000000.0,2017,2016,Kostas Mitroglou,gr GRE,...,2221.666667,17.333333,3.000000,0.333333,3.000000,0.000000,0.700000,0.120000,0.820000,2016
71,Pawel Dawidowicz,24.0,Centre-Back,Hellas Verona,Summer,3500000.0,2019,2018,Eduardo Salvio,ar ARG,...,1097.052838,4.125245,2.956947,0.000000,1.260274,0.000000,0.299393,0.236223,0.538043,2018


Using fuzzy we were able to correctly join rows 1,2,4,5. Let's add these rows to our first merged dataframe.

In [12]:
#Selecting only the rows fuzzy was able to join correctly
double_merged_df = double_merged_df.iloc[[1,2,4,5]]

In [13]:
#Adding them to merged_df
merged_df = pd.concat([merged_df, double_merged_df], ignore_index=True)

In [14]:
#Cleaning merged_df of unneccesary columns
merged_df.drop(['Player','Nation', 'Pos', 'Age','Season','stats_year_summer'],axis=1,inplace=True)

### Merging with ContractDate_df


In [15]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   player_name         46 non-null     object 
 1   age                 46 non-null     float64
 2   position            46 non-null     object 
 3   club_involved_name  46 non-null     object 
 4   transfer_period     46 non-null     object 
 5   fee_cleaned         46 non-null     float64
 6   year                46 non-null     int64  
 7   cum_MP              46 non-null     float64
 8   cum_Min             46 non-null     float64
 9   cum_Gls             46 non-null     float64
 10  cum_Ast             46 non-null     float64
 11  cum_PK              46 non-null     float64
 12  cum_CrdY            46 non-null     float64
 13  cum_CrdR            46 non-null     float64
 14  cum_GlsxMin         46 non-null     float64
 15  cum_AstxMin         46 non-null     float64
 16  cum_G+AxMi

In [16]:
contractDate_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276 entries, 0 to 275
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   player_name     276 non-null    object 
 1   tf_season       276 non-null    int64  
 2   days_remaining  276 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.6+ KB


In [17]:
full_df = pd.merge(merged_df, contractDate_df, left_on=['player_name','year'], right_on=['player_name','tf_season'], how='left')


In [18]:
full_df.columns

Index(['player_name', 'age', 'position', 'club_involved_name',
       'transfer_period', 'fee_cleaned', 'year', 'cum_MP', 'cum_Min',
       'cum_Gls', 'cum_Ast', 'cum_PK', 'cum_CrdY', 'cum_CrdR', 'cum_GlsxMin',
       'cum_AstxMin', 'cum_G+AxMin', 'tf_season', 'days_remaining'],
      dtype='object')

In [19]:
full_df.drop('tf_season',axis=1,inplace=True)

In [20]:
full_df['days_remaining'].isna().sum()

11

In [25]:
transfers_df[['player_name']].to_csv('C:/Users/39329/Documents/GitHub/Football_players_trasfer_value/Test/transfer_names.csv')

In [27]:
stats_df[['Player']].to_csv('C:/Users/39329/Documents/GitHub/Football_players_trasfer_value/Test/stats_names.csv')